<a href="https://colab.research.google.com/github/chaithanyatenneti/Cloud_Detection/blob/main/Cloud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

In [6]:
true_folder = 'Data/Train/true'
false_folder = 'Data/Train/false'

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
class ImageFolder(Dataset):
    def __init__(self, root, transform=None, is_true=True):
        self.root = root
        self.transform = transform
        self.is_true = is_true
        self.image_files = [f for f in os.listdir(root) if f.endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = 1 if self.is_true else 0  # 1 for true, 0 for false
        return image, label

true_dataset = ImageFolder(true_folder, transform=transform, is_true=True)
false_dataset = ImageFolder(false_folder, transform=transform, is_true=False)

dataset = torch.utils.data.ConcatDataset([true_dataset, false_dataset])


In [9]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [10]:
len(train_loader)

28

In [11]:
len(val_loader)

7

In [12]:
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
model = models.resnet50(pretrained = True)


c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 1)

In [14]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [16]:
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.unsqueeze(1).float().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/15, Loss: 0.41427455842494965
Epoch 2/15, Loss: 0.3369898311793804
Epoch 3/15, Loss: 0.27995178954941885
Epoch 4/15, Loss: 0.27777100008513245
Epoch 5/15, Loss: 0.25935823363917215
Epoch 6/15, Loss: 0.2536276892891952
Epoch 7/15, Loss: 0.26417566356914385
Epoch 8/15, Loss: 0.27519760387284414
Epoch 9/15, Loss: 0.2299202031322888
Epoch 10/15, Loss: 0.2026912769568818
Epoch 11/15, Loss: 0.21795537109885896
Epoch 12/15, Loss: 0.20605838964028017
Epoch 13/15, Loss: 0.2279786885316883
Epoch 14/15, Loss: 0.21970726629453047
Epoch 15/15, Loss: 0.16244399015392577


In [17]:
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs) > 0.5
        all_preds.extend(preds.cpu().numpy().flatten())
        all_labels.extend(labels.cpu().numpy())

In [18]:
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.9227272727272727
Precision: 0.9395973154362416
Recall: 0.9459459459459459
F1 Score: 0.9427609427609427


In [19]:
test_folder = 'Data/Test'

test_dataset = ImageFolder(test_folder, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [20]:
model.eval()
test_preds = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs) > 0.5
        test_preds.extend(preds.cpu().numpy().flatten())